# Custom Validator


To make the implementation of custom validators easier, we have provided [an interface in the OSS](https://github.com/guardrails-ai/guardrails/blob/main/guardrails/validator_base.py).

There are a few key steps to get up and running with a custom validator:

1. Implementing the validator
2. Conforming to the required interface
3. Running Locally/Submitting to the Validator Hub


First, let's build out a simple custom validator. This validator will check if all text in the input is lowercase.


In [1]:
from typing import Any, Dict
import requests
from guardrails.validator_base import (
    FailResult,
    PassResult,
    ValidationResult,
    Validator,
    register_validator,
    ErrorSpan,
)


@register_validator(name="guardrails/lower_case", data_type="string")
class LowercaseValidator(Validator):
    def __init__(
        self,
    ):
        self.rail_alias = "lowercase"

    def _validate(self, value: Any, metadata: Dict[str, Any]) -> ValidationResult:
        if not isinstance(value, str):
            return FailResult(error_message="Input must be a string.", fix_value=None)

        inference_result = self._inference(value)

        if inference_result:
            return PassResult()
        else:
            return FailResult(
                error_message="Input must be lowercase.", fix_value=value.lower()
            )

    def _inference_local(self, model_input: str) -> bool:
        """Implement a function to perform inference on a local machine."""
        return model_input.islower()

    def _inference_remote(self, model_input: str) -> bool:
        """Implement a function that will build a request and perform inference on a
        remote machine. This is not required if you will always use local mode.
        """
        response = requests.post(self.validation_endpoint, json={"inputs": model_input})
        if response.status_code != 200:
            raise Exception(
                f"Remote inference failed with status code {response.status_code}"
            )

        return response.json().get("is_lowercase", False)

# Simple usage running locally

If this validator was stored locally in your codebase, you would want to import it

```bash
from lowercase_validator import LowercaseValidator
```


In [2]:
# Usage example:
lowercase_validator_local = LowercaseValidator(use_local=True)
lowercase_validator_remote = LowercaseValidator(
    use_local=False, validation_endpoint="http://example.com/validate"
)

# Test cases for local validator
print(lowercase_validator_local("hello world"))  # PassResult
print(lowercase_validator_local("Hello World"))  # FailResult
print(lowercase_validator_local("123"))  # PassResult (numbers are considered lowercase)
print(lowercase_validator_local(123))  # FailResult (not a string)

TypeError: LowercaseValidator.__init__() got an unexpected keyword argument 'use_local'

The best practice would be to use the validator in a Guardrails.Guard()


In [ ]:
# Make some imports
from rich import print
import guardrails as gd
import litellm
from IPython.display import clear_output
import time

In [ ]:
prompt = "Tell me about the company Apple. Make sure the output is all in lowercase. Don't use any capital letters."

guard = gd.Guard().use(LowercaseValidator)

Instantiate a guard object


In [ ]:
result = fragment_generator = guard(
    litellm.completion,
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Tell me about LLM streaming APIs."},
    ],
    max_tokens=1024,
    temperature=0,
)

print(result)

If you would like an improved implementation, you can implement the new ErrorSpan feature. This class provides a way to define the span of the error in the input.


In [ ]:
@register_validator(name="guardrails/lower_case", data_type="string")
class LowercaseValidator(Validator):
    def __init__(
        self,
    ):
        self.rail_alias = "lowercase"

    def _validate(self, value: Any, metadata: Dict[str, Any]) -> ValidationResult:
        if not isinstance(value, str):
            return FailResult(error_message="Input must be a string.", fix_value=None)

        inference_result = self._inference(value)

        if inference_result:
            return PassResult()
        else:
            return FailResult(
                error_message="Input must be lowercase.", fix_value=value.lower()
            )

    def _inference_local(self, model_input: str) -> bool:
        """Implement a function to perform inference on a local machine."""
        error_spans = []
        start = None

        for i, char in enumerate(model_input):
            if char.isupper():
                if start is None:
                    start = i
            elif start is not None:
                error_spans.append(ErrorSpan(start, i - 1))
                start = None

        if start is not None:
            error_spans.append(ErrorSpan(start, len(model_input) - 1))

        return error_spans

    def _inference_remote(self, model_input: str) -> bool:
        """Implement a function that will build a request and perform inference on a
        remote machine. This is not required if you will always use local mode.
        """
        response = requests.post(self.validation_endpoint, json={"inputs": model_input})
        if response.status_code != 200:
            raise Exception(
                f"Remote inference failed with status code {response.status_code}"
            )

        return response.json().get("is_lowercase", False)

In [ ]:
result = fragment_generator = guard(
    litellm.completion,
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Tell me about LLM streaming APIs."},
    ],
    max_tokens=1024,
    temperature=0,
)

print(result)